Preface:
    The classic Black Scholes formula as implemented below assumes:
        1) The amount of time that is used for the determining the distribution of the underlying value 
        at expiry is the same amount of time that should be used for discounting the forward price of 
        the option.  Frequently these two amounts of time are different as options rarely settle at expiry
        but instead settle at expiry plus a few days.
        2) The interest rate used to project the underlying value at expiry based on the underlying spot 
        value is the same as the interest rate that should be used for discounting the forward price of
        the option.  In crytpo markets in 2025, for example, these two rates are very different.
    To overcome these shortcomings, the following implementation is recommended:
        1) Set the interest rate in the Black Scholes formula to zero.
        2) Calculate the underlying value at expiry prior to calling the formula.  Enter the underlying
        value at epxiry as the undelrying_value variable# Assistant
        3) Calculate the amount of time to expiration of the option prior to calling the formual.  Set
        the time variable equal to this amount.
        4) Discount the formulas's resulting forward price of the option after using the formula below
        using the Time_Value_Money Class and choices for rate and time that are specific to the discounting
        situation (time to payment rather than time to expiry and rate for discounting rather than forward
        rate of underlying).
    The term "value" is used rather than "price" to emphasize that formulas below can be used for rates, too

Parameters:
    c_or_p (string): acceptable names for 'call' or 'put' (see c_or_p function below)
    u (float): see note above, value of underlying
    k (float): strike value expressed in same units as underlying_value
    v (float): volatility expressed as a percentage of underlying value (should be the same value regardless of 
        using forward or spot underlying price)
    t (float): see note above, fraction of a year (use Daycount_Math prior to calling functions below)
    r (float): see note above, continuously compounded rate used to (i) discount forward option 
        value to present value and (ii) create underlying value at expiry (if spot underlying value is used
        as the input) 
    d0 (float): the amount that is directly in the middle of d1 and d2 - 
        similar to z-score of u relative to k based on v * sqrt(t)
    d1 (float): the amount equal to the d1 term in the Black Scholes formula
    d2 (float): the amount equal to the d2 term in the Black Scholes formula
    opt_value (float): the value of the option in the same units as the units of the underlying

Results:
    d0 : same as in parameters
    d1 : same as in parameters
    d2 : same as in parameters
    option_value : value of option in same units as underlying value
    delta : change in option_value for a one unit change in underlying value (first order greek)
    vega : change in option_value for a one percent absolute change in volatility (first order greek)
    theta : change in option_value for a one day change in time (first order greek)
    rho : change in option_value for a one percent absolute change in rate (first order greek)
    gamma : change in delta for a one unit change in underlying value (second order greek)
    volga : change in vega for a one percent absolute value change in volatility (second order greek)
    speed : (third order greek)
    ultima : (third order greek)
    vanna : (cross greek)
    charm : (cross greek)
    veta : (cross greek)
    zoma : (cross greek)
    color : (cross greek)
    

In [ ]:
import numpy as np 
from scipy.stats import norm
from scipy.optimize import brentq

In [1]:
class OptionsMathBlackScholes(OptionsMathHelpers):

    
    def __init__(self):
        # constructor 
        pass

    
    # -----------------------
    # Classic Black-Scholes Formulas
    #      Formulas just calculate answers 
    #      Formulas should be good for use with single inputs or arrays
    #      All data should be checked before using formulas as formulas assume data is consistent and clean
    #      (self, c_or_p='cp', u='u', k='k', v='v', t='t', r='r', d0='d0', d1='d1', d2='d2', diy='diy', **kwargs):
    # -----------------------
        
    def bs_d0_formula(self, u='u', k='k', v='v', t='t', r=0.0, **kwargs): 
        # Prevent division by zero or invalid sqrt(time)
        t = np.maximum(t, 1e-12)
        v = np.maximum(v, 1e-12)
        vol_sqrt_time = v * np.sqrt(t)  
        log_moneyness = np.log(u / k)
        growth_rate   = r * t         
        return (log_moneyness + growth_rate) / vol_sqrt_time

    
    def bs_d1_formula(self, v='v', t='t', d0='d0', **kwargs):
        return d0 + (v * np.sqrt(t) / 2.0)

    
    def bs_d2_formula(self, v='v', t='t', d0='d0', **kwargs):
        return d0 - (v * np.sqrt(t) / 2.0)

    
    def bs_option_value_formula(self, c_or_p='cp', u='u', k='k', t='t', r=0.0, d1='d1', d2='d2', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        u_term  =  scalar * norm.cdf(d1 * scalar) *  u 
        k_term  = -scalar * norm.cdf(d2 * scalar) * (k * self.pv_factor(t=t, r=r))
        return u_term + k_term         
        

    def bs_delta_formula(self, c_or_p='cp', d1='d1', **kwargs):
        call_delta = norm.cdf(d1)
        put_delta  = call_delta - 1  # N(d1) − 1 = −N(−d1)
        return np.where(c_or_p == 'CALL', call_delta, put_delta)

    
    def bs_vega_formula(self, u='u', t='t', d1='d1', **kwargs):
        return u * np.sqrt(t) * norm.pdf(d1) * 0.01

    
    def bs_theta_formula(self, c_or_p='cp', u='u', k='k', v='v', t='t', r=0.0, d1='d1', d2='d2', diy='diy', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        u_term  = -1 * u * v * norm.pdf(d1) / (2 * np.sqrt(t))
        k_term  = -1 * k * r * norm.cdf(d2 * scalar) * self.pv_factor(t=t, r=r) * scalar         
        return (u_term + k_term) / diy
        

    def bs_rho_formula(self, c_or_p='cp', k='k', t='t', r=0.0, d2='d2', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        return k * t * norm.cdf(d2 * scalar) * self.pv_factor(t=t, r=r) * scalar * 0.01
        

    def bs_gamma_formula(self, u='u', v='v', t='t', d1='d1', **kwargs):
        denominator = u * v * np.sqrt(t)
        return norm.pdf(d1) / denominator  


    def bs_volga_formula(self, v='v', d1='d1', d2='d2', vega='v', **kwargs):
        numerator = vega * d1 * d2
        return numerator / v

    
    # -----------------------
    # Option formulas with data checks
    #      Formulas can be called with multiple input combinations (with or without d-terms, etc.)
    #      Formulas convert inputs to NumPy arrays for elementwise math
    #      Formulas should also work with non-array inputs
    #      (self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, d1=-100.0, d2=-100.0, diy=-100.0, **kwargs):
    # -----------------------        
        
    def bs_d0(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, **kwargs):
        u, k, t, v, r = self.to_arrays(u, k, t, v, r)
        return self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        

    def bs_d1(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, **kwargs):
        u, k, t, v, r, d0 = self.to_arrays(u, k, t, v, r, d0)   
        if np.all(d0 == -100.0):
            d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        return self.bs_d1_formula(d0=d0, v=v, t=t)

    
    def bs_d2(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, **kwargs):
        u, k, t, v, r, d0 = self.to_arrays(u, k, t, v, r, d0)   
        if np.all(d0 == -100.0):
            d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        return self.bs_d2_formula(d0=d0, v=v, t=t)

       
    def bs_option_value(self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, 
                        d0=-100.0, d1=-100.0, d2=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)
        return self.bs_option_value_formula(c_or_p=c_or_p, u=u, k=k, t=t, r=r, d1=d1, d2=d2)   

    
    # -----------------------
    # Greeks (first order)
    # -----------------------
    def bs_delta(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, d1=-100.0,  **kwargs):  
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        c_or_p = self.c_or_p(c_or_p, d1)
        return self.bs_delta_formula(c_or_p=c_or_p, d1=d1)
        

    def bs_vega(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, d1=-100.0,  **kwargs):  
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        return self.bs_vega_formula(u=u, t=t, d1=d1)

    
    def bs_theta(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, 
                 d0=-100.0, d1=-100.0, d2=-100.0, diy=365, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)                 
        return self.bs_theta_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1, d2=d2, diy=diy)


    def bs_rho(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, d2=-100.0, **kwargs):
        u, k, v, t, r, d0, d2 = self.to_arrays(u, k, v, t, r, d0, d2)  
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, k)       
        return self.bs_rho_formula(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2) 

    
    # -----------------------
    # Greeks (second order)
    # -----------------------
    def bs_gamma(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, d0=-100.0, d1=-100.0, **kwargs):        
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if d1 == [-100]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)         
        return self.bs_gamma_formula(u=u, v=v, t=t, d1=d1)

        
    def bs_volga(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, 
                 d0=-100.0, d1 = -100.0, d2=-100.0, vega=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if d1 == [-100.0]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if d2 == [-100]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)     
        if vega == [-100]:
            vega = self.bs_vega_formula(u=u, t=t, d1=d1)
        return self.bs_volga_formula(v=v, d1=d1, d2=d2, vega=vega)


    # -----------------------
    # Greeks (third order)
    # -----------------------
    def bs_speed():
        pass
        #pending

    
    def bs_ultima():
        pass
        #pending        


    # -----------------------
    # Greeks (cross)
    # -----------------------
    def bs_vanna():
        pass
        #pending


    def bs_charm():
        pass
        #pending


    def bs_veta():
        pass
        #pending


    def bs_zomma():
        pass
        #pending


    def bs_color():
        pass
        #pending

    # -----------------------
    # Combinations of greeks
    # ----------------------- 
    def bs_first_order_greeks(self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0.0, 
                              d0=-100.0, d1=-100.0, d2=-100.0, diy=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2, diy = self.to_arrays(u, k, v, t, r, d0, d1, d2, diy) 
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)  
        delta  = self.bs_delta_formula(c_or_p=c_or_p, d1=d1)
        vega   = self.bs_vega_formula(u=u, t=t, d1=d1)
        theta  = self.bs_theta_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1, d2=d2, diy=diy)
        rho    = self.bs_rho_formula(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2)
        return (delta, vega, theta, rho)

    
    def bs_second_order_greeks():
        pass


    def bs_third_order_greeks():
        pass


    def bs_cross_greeks():
        pass
                

    def bs_all_greeks():
        pass

        
    # -----------------------
    # Volatility backsolvers
    # ----------------------- 
    def bs_vol_solver_brentq(self, opt_value=-100.0, c_or_p='', u=-100.0, k=-100.0, t=-100.0, r=0.0):
        """
        Solve for implied volatility given a market price.
        """
        
        u, k, t, r, opt_value = self.to_arrays(u, k, t, r, opt_value)  
        c_or_p = self.c_or_p(c_or_p, u)

        lo_vol_bound = np.full(opt_value.shape, 1e-6)
        hi_vol_bound = np.full(opt_value.shape, 10.0)     
        
        # Define the function whose root we want to find
        def objective(v):
            d0 = self.bs_d0_formula(u=u, k=k, v=v, t=t, r=r)
            d1 = self.bs_d1_formula(v=v, t=t, d0=d0)
            d2 = self.bs_d2_formula(v=v, t=t, d0=d0)
            return self.bs_option_value_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r) - opt_value
    
        # Use brentq to find sigma between lo_vol_bound and hi_vol_bound
        try:
            return brentq(objective, lo_vol_bound, hi_vol_bound)
        except ValueError:
            return np.nan  # If no solution in the range


    def bs_vol_solver_newton(self, opt_value=-100.0, c_or_p='', u=-100.0, k=-100.0, t=-100.0, r=0,
                             initial_guess=0.5, tolerance=1e-8, max_iterations=100):
        """
        Compute implied volatility using Newton–Raphson iteration.
        """
        v = initial_guess

        u, k, v, t, r, opt_value = self.to_arrays(u, k, v, t, r, opt_value)  
        c_or_p = self.c_or_p(c_or_p, u)

        for i in range(max_iterations):
            # Avoid division by zero or tiny Vega
            if np.any(v < 1e-8):
                break
                
            d0 = self.bs_d0_formula(u=u, k=k, v=v, t=t, r=r)
            d1 = self.bs_d1_formula(v=v, t=t, d0=d0)
            d2 = self.bs_d2_formula(v=v, t=t, d0=d0)
            opt_value_est = self.bs_option_value_formula(c_or_p=c_or_p, u=u, k=k, t=t, r=r, d1=d1, d2=d2)
            vega = self.bs_vega_formula(u=u, t=t, d1=d1)
       
            diff = opt_value_est - opt_value
            v = v - (diff / vega)  # Newton-Raphson step
    
            # Convergence check
            if np.all(abs(diff) < tolerance):
                return v
    
        return np.nan  # Did not converge

